<a href="https://colab.research.google.com/github/fangminnnnnnnnnnn/translations/blob/main/Translations_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from transformers import pipeline
import csv

# 初始化翻譯模型
fr_to_zh_translator = pipeline("translation", model="facebook/m2m100_418M")  # 法文到中文

# 讀取法文原文文件
input_file = "original_texts.txt"
french_texts = []
with open(input_file, "r", encoding="utf-8") as file:
    french_texts = [line.strip() for line in file if line.strip()]  # 移除空行

# 保存翻譯結果的文件名
output_file = "generated_translations.csv"

# 存儲翻譯結果
translations = []

# 翻譯每個法文原文
for french_text in french_texts:
    print(f"處理原文：{french_text}")

    # 標準翻譯
    standard_translation = fr_to_zh_translator(french_text, src_lang="fr", tgt_lang="zh")[0]['translation_text']

    # 詩意翻譯
    poetic_translation = fr_to_zh_translator("Poetic style: " + french_text, src_lang="fr", tgt_lang="zh")[0]['translation_text']

    # 口語化翻譯
    colloquial_translation = fr_to_zh_translator("Colloquial style: " + french_text, src_lang="fr", tgt_lang="zh")[0]['translation_text']

    # 保存翻譯結果
    translations.append({
        "原文": french_text,
        "標準翻譯": standard_translation,
        "詩意翻譯": poetic_translation,
        "口語化翻譯": colloquial_translation
    })

# 將翻譯結果保存到 CSV 文件
with open(output_file, "w", newline="", encoding="utf-8") as csvfile:
    fieldnames = ["原文", "標準翻譯", "詩意翻譯", "口語化翻譯"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(translations)

print(f"翻譯結果已保存到文件：{output_file}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/908 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.94G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/233 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/298 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/3.71M [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

Device set to use cpu


處理原文：原文
處理原文：« L’art de l’apprivoisement : la création de liens véritables »
處理原文：« Adieu, dit le renard. Voici mon secret. Il est très simple : on ne voit bien qu'avec le cœur. L'essentiel est invisible pour les yeux. »
處理原文：« L'essentiel est invisible pour les yeux, » répéta le petit prince, afin de se souvenir.
處理原文：« C'est le temps que tu as perdu pour ta rose qui fait ta rose si importante. »
處理原文：« C'est le temps que j'ai perdu pour ma rose... » fit le petit prince, afin de se souvenir.
處理原文：« Les hommes ont oublié cette vérité, dit le renard. Mais tu ne dois pas l'oublier. Tu deviens responsable pour toujours de ce que tu as apprivoisé. Tu es responsable de ta rose... »
處理原文：« Je suis responsable de ma rose... » répéta le petit prince, afin de se souvenir.
處理原文：« La rose et le petit prince : un lien unique et éternel »
處理原文：« Je croyais être riche d’une fleur unique, et je ne possède qu’une rose ordinaire.
處理原文：Ça et mes trois volcans qui m’arrivent aux genoux, et dont l’un peut

In [ ]:
!pip install rouge-score


  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=f0fb7b2f54f6cea4106a8f99f257c10f4e1e0140f8c265d77f4dfd398ae3a2e8
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [ ]:
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer
import csv

# 定義參考翻譯文件和生成翻譯文件
reference_file = "reference_translations.csv"  # 參考翻譯文件
generated_file = "generated_translations.csv"  # 生成的翻譯文件
output_scores_file = "translation_scores.csv"  # 評分結果輸出文件

# 初始化 ROUGE 計算器
scorer = rouge_scorer.RougeScorer(["rougeL"], use_stemmer=True)

# 讀取參考翻譯
references = []
with open(reference_file, "r", encoding="utf-8") as file:
    for line in file:
        parts = line.strip().split("|")
        if len(parts) == 4:  # 確保有四部分
            references.append({
                "原文": parts[0],
                "標準翻譯": parts[1],
                "詩意翻譯": parts[2],
                "口語化翻譯": parts[3]
            })

# 讀取生成的翻譯
generated_translations = []
with open(generated_file, "r", encoding="utf-8") as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        generated_translations.append(row)

# 確保數據對應
if len(references) != len(generated_translations):
    print("參考翻譯與生成翻譯的數量不匹配！")
    exit()

# 計算評分
scores = []
for ref, gen in zip(references, generated_translations):
    try:
        # BLEU 和 ROUGE 計算（標準翻譯）
        bleu_standard = sentence_bleu([ref["標準翻譯"].split()], gen["標準翻譯"].split())
        rouge_standard = scorer.score(ref["標準翻譯"], gen["標準翻譯"])["rougeL"].fmeasure

        # BLEU 和 ROUGE 計算（詩意翻譯）
        bleu_poetic = sentence_bleu([ref["詩意翻譯"].split()], gen["詩意翻譯"].split())
        rouge_poetic = scorer.score(ref["詩意翻譯"], gen["詩意翻譯"])["rougeL"].fmeasure

        # BLEU 和 ROUGE 計算（口語化翻譯）
        bleu_colloquial = sentence_bleu([ref["口語化翻譯"].split()], gen["口語化翻譯"].split())
        rouge_colloquial = scorer.score(ref["口語化翻譯"], gen["口語化翻譯"])["rougeL"].fmeasure

        # 保存分數
        scores.append({
            "原文": ref["原文"],
            "標準翻譯 BLEU": bleu_standard,
            "標準翻譯 ROUGE-L": rouge_standard,
            "詩意翻譯 BLEU": bleu_poetic,
            "詩意翻譯 ROUGE-L": rouge_poetic,
            "口語化翻譯 BLEU": bleu_colloquial,
            "口語化翻譯 ROUGE-L": rouge_colloquial
        })
    except Exception as e:
        print(f"評分過程出錯：{e}")

# 將評分結果保存到 CSV 文件
with open(output_scores_file, "w", newline="", encoding="utf-8") as csvfile:
    fieldnames = [
        "原文",
        "標準翻譯 BLEU", "標準翻譯 ROUGE-L",
        "詩意翻譯 BLEU", "詩意翻譯 ROUGE-L",
        "口語化翻譯 BLEU", "口語化翻譯 ROUGE-L"
    ]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(scores)

print(f"評分結果已保存到文件：{output_scores_file}")

參考翻譯與生成翻譯的數量不匹配！
評分結果已保存到文件：translation_scores.csv


In [ ]:
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer
import csv

# 定義文件路徑
reference_file = "reference_translations.csv"  # 參考翻譯文件
generated_file = "generated_translations.csv"  # 生成的翻譯文件
output_scores_file = "translation_scores.csv"  # 評分結果輸出文件

# 初始化 ROUGE 計算器
scorer = rouge_scorer.RougeScorer(["rougeL"], use_stemmer=True)

# 讀取參考翻譯
references = []
with open(reference_file, "r", encoding="utf-8") as file:
    reader = csv.DictReader(file)
    for row in reader:
        references.append({
            "原文": row["原文"],
            "標準翻譯": row["標準翻譯"],
            "詩意翻譯": row["詩意翻譯"],
            "口語化翻譯": row["口語化翻譯"]
        })

# 讀取生成的翻譯
generated_translations = []
with open(generated_file, "r", encoding="utf-8") as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        generated_translations.append(row)

# 確保數據對應
if len(references) != len(generated_translations):
    print("參考翻譯與生成翻譯的數量不匹配！")
    exit()

# 計算評分
scores = []
for ref, gen in zip(references, generated_translations):
    try:
        # BLEU 和 ROUGE 計算（標準翻譯）
        bleu_standard = sentence_bleu([ref["標準翻譯"].split()], gen["標準翻譯"].split())
        rouge_standard = scorer.score(ref["標準翻譯"], gen["標準翻譯"])["rougeL"].fmeasure

        # BLEU 和 ROUGE 計算（詩意翻譯）
        bleu_poetic = sentence_bleu([ref["詩意翻譯"].split()], gen["詩意翻譯"].split())
        rouge_poetic = scorer.score(ref["詩意翻譯"], gen["詩意翻譯"])["rougeL"].fmeasure

        # BLEU 和 ROUGE 計算（口語化翻譯）
        bleu_colloquial = sentence_bleu([ref["口語化翻譯"].split()], gen["口語化翻譯"].split())
        rouge_colloquial = scorer.score(ref["口語化翻譯"], gen["口語化翻譯"])["rougeL"].fmeasure

        # 保存分數
        scores.append({
            "原文": ref["原文"],
            "標準翻譯 BLEU": bleu_standard,
            "標準翻譯 ROUGE-L": rouge_standard,
            "詩意翻譯 BLEU": bleu_poetic,
            "詩意翻譯 ROUGE-L": rouge_poetic,
            "口語化翻譯 BLEU": bleu_colloquial,
            "口語化翻譯 ROUGE-L": rouge_colloquial
        })
    except Exception as e:
        print(f"評分過程出錯：{e}")

# 將評分結果保存到 CSV 文件
with open(output_scores_file, "w", newline="", encoding="utf-8") as csvfile:
    fieldnames = [
        "原文",
        "標準翻譯 BLEU", "標準翻譯 ROUGE-L",
        "詩意翻譯 BLEU", "詩意翻譯 ROUGE-L",
        "口語化翻譯 BLEU", "口語化翻譯 ROUGE-L"
    ]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(scores)

print(f"評分結果已保存到文件：{output_scores_file}")


KeyError: '標準翻譯'

In [ ]:
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer
import csv

# 定義文件路徑
reference_file = "reference_translations.csv"  # 參考翻譯文件
generated_file = "generated_translations.csv"  # 生成的翻譯文件
output_scores_file = "translation_scores.csv"  # 評分結果輸出文件

# 初始化 ROUGE 計算器
scorer = rouge_scorer.RougeScorer(["rougeL"], use_stemmer=True)

# 讀取參考翻譯
references = []
with open(reference_file, "r", encoding="utf-8") as file:
    reader = csv.DictReader(file)
    for row in reader:
        references.append({
            "原文": row["原文"],
            "標準翻譯": row["標準翻譯_reference"],
            "詩意翻譯": row["詩意翻譯_reference"],
            "口語化翻譯": row["口語化翻譯_reference"]
        })

# 讀取生成的翻譯
generated_translations = []
with open(generated_file, "r", encoding="utf-8") as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        generated_translations.append(row)

# 確保數據對應
if len(references) != len(generated_translations):
    print("參考翻譯與生成翻譯的數量不匹配！")
    exit()

# 計算評分
scores = []
for ref, gen in zip(references, generated_translations):
    try:
        # BLEU 和 ROUGE 計算（標準翻譯）
        bleu_standard = sentence_bleu([ref["標準翻譯"].split()], gen["標準翻譯"].split())
        rouge_standard = scorer.score(ref["標準翻譯"], gen["標準翻譯"])["rougeL"].fmeasure

        # BLEU 和 ROUGE 計算（詩意翻譯）
        bleu_poetic = sentence_bleu([ref["詩意翻譯"].split()], gen["詩意翻譯"].split())
        rouge_poetic = scorer.score(ref["詩意翻譯"], gen["詩意翻譯"])["rougeL"].fmeasure

        # BLEU 和 ROUGE 計算（口語化翻譯）
        bleu_colloquial = sentence_bleu([ref["口語化翻譯"].split()], gen["口語化翻譯"].split())
        rouge_colloquial = scorer.score(ref["口語化翻譯"], gen["口語化翻譯"])["rougeL"].fmeasure

        # 保存分數
        scores.append({
            "原文": ref["原文"],
            "標準翻譯 BLEU": bleu_standard,
            "標準翻譯 ROUGE-L": rouge_standard,
            "詩意翻譯 BLEU": bleu_poetic,
            "詩意翻譯 ROUGE-L": rouge_poetic,
            "口語化翻譯 BLEU": bleu_colloquial,
            "口語化翻譯 ROUGE-L": rouge_colloquial
        })
    except Exception as e:
        print(f"評分過程出錯：{e}")

# 將評分結果保存到 CSV 文件
with open(output_scores_file, "w", newline="", encoding="utf-8") as csvfile:
    fieldnames = [
        "原文",
        "標準翻譯 BLEU", "標準翻譯 ROUGE-L",
        "詩意翻譯 BLEU", "詩意翻譯 ROUGE-L",
        "口語化翻譯 BLEU", "口語化翻譯 ROUGE-L"
    ]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(scores)

print(f"評分結果已保存到文件：{output_scores_file}")


參考翻譯與生成翻譯的數量不匹配！
評分結果已保存到文件：translation_scores.csv


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

In [ ]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer
import csv

# 定義文件路徑
reference_file = "reference_translations.csv"  # 參考翻譯文件
generated_file = "cleaned_generated_translations.csv"  # 生成的翻譯文件
output_scores_file = "translation_scores.csv"  # 評分結果輸出文件

# 初始化平滑函數和 ROUGE 計算器
smoothing = SmoothingFunction()
scorer = rouge_scorer.RougeScorer(["rougeL"], use_stemmer=True)

# 讀取參考翻譯
references = []
with open(reference_file, "r", encoding="utf-8") as file:
    reader = csv.DictReader(file)
    for row in reader:
        references.append({
            "原文": row["原文"],
            "標準翻譯": row["標準翻譯_reference"],
            "詩意翻譯": row["詩意翻譯_reference"],
            "口語化翻譯": row["口語化翻譯_reference"]
        })

# 讀取生成的翻譯
generated_translations = []
with open(generated_file, "r", encoding="utf-8") as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        generated_translations.append(row)

# 確保數據對應
if len(references) != len(generated_translations):
    print("參考翻譯與生成翻譯的數量不匹配！")
    exit()

# 計算評分
scores = []
for ref, gen in zip(references, generated_translations):
    try:
        # BLEU 和 ROUGE 計算（標準翻譯）
        bleu_standard = sentence_bleu(
            [ref["標準翻譯"].split()], gen["標準翻譯"].split(), smoothing_function=smoothing.method1
        )
        rouge_standard = scorer.score(ref["標準翻譯"], gen["標準翻譯"])["rougeL"].fmeasure

        # BLEU 和 ROUGE 計算（詩意翻譯）
        bleu_poetic = sentence_bleu(
            [ref["詩意翻譯"].split()], gen["詩意翻譯"].split(), smoothing_function=smoothing.method1
        )
        rouge_poetic = scorer.score(ref["詩意翻譯"], gen["詩意翻譯"])["rougeL"].fmeasure

        # BLEU 和 ROUGE 計算（口語化翻譯）
        bleu_colloquial = sentence_bleu(
            [ref["口語化翻譯"].split()], gen["口語化翻譯"].split(), smoothing_function=smoothing.method1
        )
        rouge_colloquial = scorer.score(ref["口語化翻譯"], gen["口語化翻譯"])["rougeL"].fmeasure

        # 保存分數
        scores.append({
            "原文": ref["原文"],
            "標準翻譯 BLEU": bleu_standard,
            "標準翻譯 ROUGE-L": rouge_standard,
            "詩意翻譯 BLEU": bleu_poetic,
            "詩意翻譯 ROUGE-L": rouge_poetic,
            "口語化翻譯 BLEU": bleu_colloquial,
            "口語化翻譯 ROUGE-L": rouge_colloquial
        })
    except Exception as e:
        print(f"評分過程出錯：{e}")
        scores.append({
            "原文": ref["原文"],
            "標準翻譯 BLEU": None,
            "標準翻譯 ROUGE-L": None,
            "詩意翻譯 BLEU": None,
            "詩意翻譯 ROUGE-L": None,
            "口語化翻譯 BLEU": None,
            "口語化翻譯 ROUGE-L": None,
            "Error": str(e)
        })

# 將評分結果保存到 CSV 文件
with open(output_scores_file, "w", newline="", encoding="utf-8") as csvfile:
    fieldnames = [
        "原文",
        "標準翻譯 BLEU", "標準翻譯 ROUGE-L",
        "詩意翻譯 BLEU", "詩意翻譯 ROUGE-L",
        "口語化翻譯 BLEU", "口語化翻譯 ROUGE-L",
        "Error"
    ]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(scores)

print(f"評分結果已保存到文件：{output_scores_file}")


參考翻譯與生成翻譯的數量不匹配！
評分結果已保存到文件：translation_scores.csv


In [ ]:
import re
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer
import csv

# 定義文件路徑
reference_file = "reference_translations.csv"
generated_file = "cleaned_generated_translations.csv"
output_scores_file = "translation_scores.csv"

# 初始化平滑函數和 ROUGE 計算器
smoothing = SmoothingFunction()
scorer = rouge_scorer.RougeScorer(["rougeL"], use_stemmer=True)

# 預處理函數
def preprocess_text(text):
    text = text.lower()  # 統一小寫
    text = re.sub(r"[^\w\s]", "", text)  # 移除標點符號
    text = text.strip()  # 去掉多餘空格
    return text

# 讀取參考翻譯
references = []
with open(reference_file, "r", encoding="utf-8") as file:
    reader = csv.DictReader(file)
    for row in reader:
        references.append({
            "原文": preprocess_text(row["原文"]),
            "標準翻譯": preprocess_text(row["標準翻譯_reference"]),
            "詩意翻譯": preprocess_text(row["詩意翻譯_reference"]),
            "口語化翻譯": preprocess_text(row["口語化翻譯_reference"])
        })

# 讀取生成的翻譯
generated_translations = []
with open(generated_file, "r", encoding="utf-8") as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        generated_translations.append({
            "原文": preprocess_text(row["原文"]),
            "標準翻譯": preprocess_text(row["標準翻譯"]),
            "詩意翻譯": preprocess_text(row["詩意翻譯"]),
            "口語化翻譯": preprocess_text(row["口語化翻譯"])
        })

# 確保數據對應
if len(references) != len(generated_translations):
    print("參考翻譯與生成翻譯的數量不匹配！")
    exit()

# 計算評分
scores = []
for ref, gen in zip(references, generated_translations):
    try:
        # BLEU 和 ROUGE 計算（標準翻譯）
        bleu_standard = sentence_bleu(
            [ref["標準翻譯"].split()], gen["標準翻譯"].split(), smoothing_function=smoothing.method4
        )
        rouge_standard = scorer.score(ref["標準翻譯"], gen["標準翻譯"])["rougeL"].fmeasure

        # BLEU 和 ROUGE 計算（詩意翻譯）
        bleu_poetic = sentence_bleu(
            [ref["詩意翻譯"].split()], gen["詩意翻譯"].split(), smoothing_function=smoothing.method4
        )
        rouge_poetic = scorer.score(ref["詩意翻譯"], gen["詩意翻譯"])["rougeL"].fmeasure

        # BLEU 和 ROUGE 計算（口語化翻譯）
        bleu_colloquial = sentence_bleu(
            [ref["口語化翻譯"].split()], gen["口語化翻譯"].split(), smoothing_function=smoothing.method4
        )
        rouge_colloquial = scorer.score(ref["口語化翻譯"], gen["口語化翻譯"])["rougeL"].fmeasure

        # 保存分數
        scores.append({
            "原文": ref["原文"],
            "標準翻譯 BLEU": bleu_standard,
            "標準翻譯 ROUGE-L": rouge_standard,
            "詩意翻譯 BLEU": bleu_poetic,
            "詩意翻譯 ROUGE-L": rouge_poetic,
            "口語化翻譯 BLEU": bleu_colloquial,
            "口語化翻譯 ROUGE-L": rouge_colloquial
        })
    except Exception as e:
        print(f"評分過程出錯：{e}")
        scores.append({
            "原文": ref["原文"],
            "標準翻譯 BLEU": None,
            "標準翻譯 ROUGE-L": None,
            "詩意翻譯 BLEU": None,
            "詩意翻譯 ROUGE-L": None,
            "口語化翻譯 BLEU": None,
            "口語化翻譯 ROUGE-L": None,
            "Error": str(e)
        })

# 將評分結果保存到 CSV 文件
with open(output_scores_file, "w", newline="", encoding="utf-8") as csvfile:
    fieldnames = [
        "原文",
        "標準翻譯 BLEU", "標準翻譯 ROUGE-L",
        "詩意翻譯 BLEU", "詩意翻譯 ROUGE-L",
        "口語化翻譯 BLEU", "口語化翻譯 ROUGE-L",
        "Error"
    ]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(scores)

print(f"評分結果已保存到文件：{output_scores_file}")


參考翻譯與生成翻譯的數量不匹配！
評分結果已保存到文件：translation_scores.csv


In [ ]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer
import re
import csv

# 定義文件路徑
reference_file = "reference_translations.csv"  # 參考翻譯文件
generated_file = "generated_translations.csv"  # 生成的翻譯文件
output_scores_file = "translation_scores.csv"  # 評分結果輸出文件

# 初始化平滑函數和 ROUGE 計算器
smoothing = SmoothingFunction()
scorer = rouge_scorer.RougeScorer(["rougeL"], use_stemmer=True)

# 預處理函數
def preprocess_text(text):
    if not text:
        return text
    text = text.lower()  # 小寫化
    text = re.sub(r"[^\w\s]", "", text)  # 移除標點符號
    text = text.strip()  # 去掉多餘空格
    return text

# 讀取參考翻譯
references = []
with open(reference_file, "r", encoding="utf-8") as file:
    reader = csv.DictReader(file)
    for row in reader:
        references.append({
            "原文": preprocess_text(row["原文"]),
            "標準翻譯": preprocess_text(row["標準翻譯_reference"]),
            "詩意翻譯": preprocess_text(row["詩意翻譯_reference"]),
            "口語化翻譯": preprocess_text(row["口語化翻譯_reference"])
        })

# 讀取生成的翻譯
generated_translations = []
with open(generated_file, "r", encoding="utf-8") as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        generated_translations.append({
            "原文": preprocess_text(row["原文"]),
            "標準翻譯": preprocess_text(row["標準翻譯"]),
            "詩意翻譯": preprocess_text(row["詩意翻譯"]),
            "口語化翻譯": preprocess_text(row["口語化翻譯"])
        })

# 確保數據對應
if len(references) != len(generated_translations):
    print("參考翻譯與生成翻譯的數量不匹配！")
    exit()

# 計算評分
scores = []
for ref, gen in zip(references, generated_translations):
    try:
        # BLEU 和 ROUGE 計算（標準翻譯）
        bleu_standard = sentence_bleu(
            [ref["標準翻譯"].split()], gen["標準翻譯"].split(), smoothing_function=smoothing.method4
        )
        rouge_standard = scorer.score(ref["標準翻譯"], gen["標準翻譯"])["rougeL"].fmeasure

        # BLEU 和 ROUGE 計算（詩意翻譯）
        bleu_poetic = sentence_bleu(
            [ref["詩意翻譯"].split()], gen["詩意翻譯"].split(), smoothing_function=smoothing.method4
        )
        rouge_poetic = scorer.score(ref["詩意翻譯"], gen["詩意翻譯"])["rougeL"].fmeasure

        # BLEU 和 ROUGE 計算（口語化翻譯）
        bleu_colloquial = sentence_bleu(
            [ref["口語化翻譯"].split()], gen["口語化翻譯"].split(), smoothing_function=smoothing.method4
        )
        rouge_colloquial = scorer.score(ref["口語化翻譯"], gen["口語化翻譯"])["rougeL"].fmeasure

        # 保存分數
        scores.append({
            "原文": ref["原文"],
            "標準翻譯 BLEU": bleu_standard,
            "標準翻譯 ROUGE-L": rouge_standard,
            "詩意翻譯 BLEU": bleu_poetic,
            "詩意翻譯 ROUGE-L": rouge_poetic,
            "口語化翻譯 BLEU": bleu_colloquial,
            "口語化翻譯 ROUGE-L": rouge_colloquial
        })
    except Exception as e:
        print(f"評分過程出錯：{e}")
        scores.append({
            "原文": ref["原文"],
            "標準翻譯 BLEU": None,
            "標準翻譯 ROUGE-L": None,
            "詩意翻譯 BLEU": None,
            "詩意翻譯 ROUGE-L": None,
            "口語化翻譯 BLEU": None,
            "口語化翻譯 ROUGE-L": None,
            "Error": str(e)
        })

# 將評分結果保存到 CSV 文件
with open(output_scores_file, "w", newline="", encoding="utf-8") as csvfile:
    fieldnames = [
        "原文",
        "標準翻譯 BLEU", "標準翻譯 ROUGE-L",
        "詩意翻譯 BLEU", "詩意翻譯 ROUGE-L",
        "口語化翻譯 BLEU", "口語化翻譯 ROUGE-L",
        "Error"
    ]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(scores)

print(f"評分結果已保存到文件：{output_scores_file}")


參考翻譯與生成翻譯的數量不匹配！
評分結果已保存到文件：translation_scores.csv


In [ ]:
pip install nltk rouge-score pandas


  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=76fe0fdc071d498a943496d7b3f22f158c60962a8e6ecaa3ea4aab0d10d6711d
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [ ]:
import nltk
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer

# 確保 NLTK 的資料已經下載
nltk.download('punkt')

# 讀取翻譯文件
def read_translation_file(file_path):
    translations = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            translations.append(line.strip())
    return translations

# 讀取參考翻譯文件
def read_reference_file(file_path):
    references = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            references.append(line.strip())
    return references

# 計算 BLEU 分數
def calculate_bleu_score(references, hypothesis):
    score = sentence_bleu([ref.split() for ref in references], hypothesis.split())
    return score

# 計算 ROUGE 分數
def calculate_rouge_score(reference, hypothesis):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = scorer.score(reference, hypothesis)
    return scores

# 主程序
def main():
    # 載入翻譯結果和參考翻譯
    translated_file = 'cleaned_generated_translations.csv'  # 替換為你的翻譯結果文件
    reference_file = 'reference_translations.csv'       # 替換為你的參考翻譯文件

    translations = read_translation_file(translated_file)
    references = read_reference_file(reference_file)

    # 檢查數量是否匹配
    if len(translations) != len(references):
        print(f"錯誤：翻譯結果數量 {len(translations)} 與參考翻譯數量 {len(references)} 不匹配！")
        return
    else:
        print(f"生成翻譯行數：{len(translations)}")
        print(f"參考翻譯行數：{len(references)}")
        print("行數一致，準備進行比較。")

    # 假設翻譯和參考翻譯是一一對應的
    for i in range(len(translations)):
        hypothesis = translations[i]
        reference = references[i]

        # 計算 BLEU 分數
        bleu_score = calculate_bleu_score([reference], hypothesis)

        # 計算 ROUGE 分數
        rouge_scores = calculate_rouge_score(reference, hypothesis)

        # 輸出結果
        print(f"翻譯結果: {hypothesis}")
        print(f"參考翻譯: {reference}")
        print(f"BLEU 分數: {bleu_score:.4f}")
        print(f"ROUGE 分數: {rouge_scores}\n")
    # 保存結果到文件
    results_file = 'translation_scores.txt'
    save_results_to_file(results, results_file)

    # 下載結果文件
    files.download(results_file)


# 執行主程序
if __name__ == "__main__":
    main()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


生成翻譯行數：124
參考翻譯行數：124
行數一致，準備進行比較。
翻譯結果: 原文,標準翻譯,詩意翻譯,口語化翻譯
參考翻譯: 原文,標準翻譯,詩意翻譯,口語化翻譯
BLEU 分數: 0.0000
ROUGE 分數: {'rouge1': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rouge2': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rougeL': Score(precision=0, recall=0, fmeasure=0)}

翻譯結果: « L’art de l’apprivoisement : la création de liens véritables »,建立真正的联系”,创造真实的联系》,The Creation of True Links
參考翻譯: "« L’art de l’apprivoisement : la création de liens véritables »","《馴養的藝術：建立真摯的聯繫》","《馴養的藝術：情感紐帶的深刻真諦》","《馴養的藝術：讓關係變得有意義》"
BLEU 分數: 0.5512
ROUGE 分數: {'rouge1': Score(precision=0.7058823529411765, recall=1.0, fmeasure=0.8275862068965517), 'rouge2': Score(precision=0.6875, recall=1.0, fmeasure=0.8148148148148148), 'rougeL': Score(precision=0.7058823529411765, recall=1.0, fmeasure=0.8275862068965517)}

翻譯結果: "« Adieu, dit le renard. Voici mon secret. Il est très simple : on ne voit bien qu'avec le cœur. L'essentiel est invisible pour les yeux. »","“再见,”狐狸说,“这是我的秘密,这是很简单的,只有心灵才能看到,最重要的是眼睛看不见。","

NameError: name 'save_results_to_file' is not defined

In [ ]:
import nltk
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer
from google.colab import files

# 確保 NLTK 的資料已經下載
nltk.download('punkt')

# 讀取翻譯文件
def read_translation_file(file_path):
    translations = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            translations.append(line.strip())
    return translations

# 讀取參考翻譯文件
def read_reference_file(file_path):
    references = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            references.append(line.strip())
    return references

# 計算 BLEU 分數
def calculate_bleu_score(references, hypothesis):
    score = sentence_bleu([ref.split() for ref in references], hypothesis.split())
    return score

# 計算 ROUGE 分數
def calculate_rouge_score(reference, hypothesis):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = scorer.score(reference, hypothesis)
    return scores

# 保存結果到文件
def save_results_to_file(results, filename):
    with open(filename, 'w', encoding='utf-8') as f:
        for result in results:
            f.write(result + "\n")

# 主程序
def main():
    # 載入翻譯結果和參考翻譯
    translated_file = 'cleaned_generated_translations.csv'  # 替換為你的翻譯結果文件
    reference_file = 'reference_translations.csv'       # 替換為你的參考翻譯文件

    translations = read_translation_file(translated_file)
    references = read_reference_file(reference_file)

    # 檢查數量是否匹配
    if len(translations) != len(references):
        print(f"錯誤：翻譯結果數量 {len(translations)} 與參考翻譯數量 {len(references)} 不匹配！")
        return
    else:
        print(f"生成翻譯行數：{len(translations)}")
        print(f"參考翻譯行數：{len(references)}")
        print("行數一致，準備進行比較。")

    results = []
    # 假設翻譯和參考翻譯是一一對應的
    for i in range(len(translations)):
        hypothesis = translations[i]
        reference = references[i]

        # 計算 BLEU 分數
        bleu_score = calculate_bleu_score([reference], hypothesis)

        # 計算 ROUGE 分數
        rouge_scores = calculate_rouge_score(reference, hypothesis)

        # 輸出結果
        result_str = (f"翻譯結果: {hypothesis}\n"
                      f"參考翻譯: {reference}\n"
                      f"BLEU 分數: {bleu_score:.4f}\n"
                      f"ROUGE 分數: {rouge_scores}\n")
        results.append(result_str)

    # 保存結果到文件
    results_file = 'translation_scores.txt'
    save_results_to_file(results, results_file)

    # 下載結果文件
    files.download(results_file)

# 執行主程序
if __name__ == "__main__":
    main()


生成翻譯行數：124
參考翻譯行數：124
行數一致，準備進行比較。


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!pip install pandas

In [ ]:
import nltk
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer
from google.colab import files

# 確保 NLTK 的資料已經下載
nltk.download('punkt')

# 讀取翻譯文件
def read_translation_file(file_path):
    translations = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            translations.append(line.strip())
    return translations

# 讀取參考翻譯文件
def read_reference_file(file_path):
    references = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            references.append(line.strip())
    return references

# 計算 BLEU 分數
def calculate_bleu_score(references, hypothesis):
    score = sentence_bleu([ref.split() for ref in references], hypothesis.split())
    return score

# 計算 ROUGE 分數
def calculate_rouge_score(reference, hypothesis):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = scorer.score(reference, hypothesis)
    return scores

# 保存結果到文件
def save_results_to_file(results, filename):
    with open(filename, 'w', encoding='utf-8') as f:
        for result in results:
            f.write(result + "\n")

# 主程序
def main():
    # 載入翻譯結果和參考翻譯
    translated_file = 'cleaned_generated_translations.csv'  # 替換為你的翻譯結果文件
    reference_file = 'reference_translations.csv'       # 替換為你的參考翻譯文件

    translations = read_translation_file(translated_file)
    references = read_reference_file(reference_file)

    # 檢查數量是否匹配
    if len(translations) != len(references):
        print(f"錯誤：翻譯結果數量 {len(translations)} 與參考翻譯數量 {len(references)} 不匹配！")
        return
    else:
        print(f"生成翻譯行數：{len(translations)}")
        print(f"參考翻譯行數：{len(references)}")
        print("行數一致，準備進行比較。")

    results = []
    # 假設翻譯和參考翻譯是一一對應的
    for i in range(len(translations)):
        hypothesis = translations[i]
        reference = references[i]

        # 計算 BLEU 分數
        bleu_score = calculate_bleu_score([reference], hypothesis)

        # 計算 ROUGE 分數
        rouge_scores = calculate_rouge_score(reference, hypothesis)

        # 輸出結果
        result_str = (f"翻譯結果: {hypothesis}\n"
                      f"參考翻譯: {reference}\n"
                      f"BLEU 分數: {bleu_score:.4f}\n"
                      f"ROUGE 分數: {rouge_scores}\n")
        results.append(result_str)

    # 保存結果到文件
    results_file = 'translation_scores.csv'
    save_results_to_file(results, results_file)

    # 下載結果文件
    files.download(results_file)

# 執行主程序
if __name__ == "__main__":
    main()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


生成翻譯行數：124
參考翻譯行數：124
行數一致，準備進行比較。


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>